# Improved Geez Keyboard

> An improved implementation of the Geez keyboard for typing in Tigrinya and Amharic

In [ ]:
#| default_exp gzkeyboard

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# Fix the import path first
import sys, os
import argparse
import time
from enum import Enum
from dataclasses import dataclass
from typing import Optional, Tuple, List, Dict
import keyboard

project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to Python path")

from nbdev.showdoc import *

## Introduction

This notebook provides documentation for the improved Geez keyboard implementation. The previous implementation had some limitations:

1. The keystroke combination processing was cumbersome and inefficient
2. The timeout handling was not optimal for user experience
3. There was limited support for different input modes and customization

## Enums and Configuration

In [ ]:
#| export
class InputMode(Enum):
    """Defines the input modes available in the Geez keyboard."""
    TIGRINYA = "tigrinya"
    AMHARIC  = "amharic"
    LATIN    = "latin"  # For regular typing without conversion

In [ ]:
#| export
@dataclass
class KeyboardConfig:
    """Configuration for the Geez keyboard input method."""
    sequence_timeout     : float         = 0.5
    default_mode         : InputMode     = InputMode.TIGRINYA
    custom_mappings_path : Optional[str] = None

In [ ]:
#| export
class KeyboardMappings:
    """Manages character mappings for different languages and input modes."""
    @staticmethod
    def get_mapping_for_mode(mode):
        """Placeholder for getting mappings by mode"""
        # Basic mapping of Latin characters to Ge'ez
        default_mapping = {
            # ሀ family (h)
            'h'  : 'ሀ',   # 1st order (ä)
            'hu' : 'ሁ',  # 2nd order (u)
            'hi' : 'ሂ',  # 3rd order (i)
            'ha' : 'ሃ',  # 4th order (a)
            'he' : 'ሄ',  # 5th order (e)
            'h\'': 'ህ', # 6th order (ə)
            'ho' : 'ሆ',  # 7th order (o)

            # ለ family (l)
            'l' : 'ለ',  'lu' : 'ሉ',   'li': 'ሊ',
            'la': 'ላ',  'le' : 'ሌ',   'l\'': 'ል',
            'lo': 'ሎ',  'lwa': 'ሏ',

            # ሐ family (hh)
            'hh': 'ሐ',  'hhu': 'ሑ',  'hhi': 'ሒ',
            'hha': 'ሓ', 'hhe': 'ሔ',  'hh\'': 'ሕ',
            'hho': 'ሖ', 'hhwa': 'ሗ',

            # መ family (m)
            'm': 'መ',   'mu': 'ሙ',   'mi': 'ሚ',
            'ma': 'ማ',  'me': 'ሜ',   'm\'': 'ም',
            'mo': 'ሞ',  'mwa': 'ሟ',

            # ሠ family (sz)
            'sz': 'ሠ',  'szu': 'ሡ',  'szi': 'ሢ',
            'sza': 'ሣ', 'sze': 'ሤ',  'sz\'': 'ሥ',
            'szo': 'ሦ', 'szwa': 'ሧ',

            # ረ family (r)
            'r': 'ረ',   'ru': 'ሩ',   'ri': 'ሪ',
            'ra': 'ራ',  're': 'ሬ',   'r\'': 'ር',
            'ro': 'ሮ',  'rwa': 'ሯ',

            # ሰ family (s)
            's': 'ሰ',   'su': 'ሱ',   'si': 'ሲ',
            'sa': 'ሳ',  'se': 'ሴ',   'ss': 'ስ',
            'so': 'ሶ',  'swa': 'ሷ',

            # ሸ family (sh)
            'sh': 'ሸ',  'shu': 'ሹ',  'shi': 'ሺ',
            'sha': 'ሻ', 'she': 'ሼ',  'sh\'': 'ሽ',
            'sho': 'ሾ', 'shwa': 'ሿ',

            # ቀ family (q)
            'q': 'ቀ',   'qu': 'ቁ',   'qi': 'ቂ',
            'qa': 'ቃ',  'qe': 'ቄ',   'q\'': 'ቅ',
            'qo': 'ቆ',  'qwa': 'ቈ',  'qwi': 'ቊ',
            'qwaa': 'ቋ','qwe': 'ቌ',  'qw': 'ቍ',

            # በ family (b)
            'b': 'በ',   'bu': 'ቡ',   'bi': 'ቢ',
            'ba': 'ባ',  'be': 'ቤ',   'b\'': 'ብ',
            'bo': 'ቦ',  'bwa': 'ቧ',

            # ተ family (t)
            't': 'ተ',   'tu': 'ቱ',   'ti': 'ቲ',
            'ta': 'ታ',  'te': 'ቴ',   't\'': 'ት',
            'to': 'ቶ',  'twa': 'ቷ',

            # ቸ family (ch)
            'ch': 'ቸ',  'chu': 'ቹ',  'chi': 'ቺ',
            'cha': 'ቻ', 'che': 'ቼ',  'ch\'': 'ች',
            'cho': 'ቾ', 'chwa': 'ቿ',

            # ኀ family (x)
            'x': 'ኀ',   'xu': 'ኁ',   'xi': 'ኂ',
            'xa': 'ኃ',  'xe': 'ኄ',   'x\'': 'ኅ',
            'xo': 'ኆ',  'xwa': 'ኈ',  'xwi': 'ኊ',
            'xwaa': 'ኋ','xwe': 'ኌ',  'xw': 'ኍ',

            # ነ family (n)
            'n': 'ነ',   'nu': 'ኑ',   'ni': 'ኒ',
            'na': 'ና',  'ne': 'ኔ',   'nn': 'ን',
            'no': 'ኖ',  'nwa': 'ኗ',

            # ኘ family (ny)
            'ny': 'ኘ',  'nyu': 'ኙ',  'nyi': 'ኚ',
            'nya': 'ኛ', 'nye': 'ኜ',  'ny\'': 'ኝ',
            'nyo': 'ኞ', 'nywa': 'ኟ',

            # አ family (a/e)
            'a': 'አ',   'u': 'ኡ',    'i': 'ኢ',
            'aa': 'ኣ',  'e': 'ኤ',    'ea': 'እ',
            'o': 'ኦ',   'oa': 'ኧ',

            # ከ family (k)
            'k': 'ከ',   'ku': 'ኩ',   'ki': 'ኪ',
            'ka': 'ካ',  'ke': 'ኬ',   'k\'': 'ክ',
            'ko': 'ኮ',  'kwa': 'ኰ',  'kwi': 'ኲ',
            'kwaa': 'ኳ','kwe': 'ኴ',  'kw': 'ኵ',

            # ኸ family (kh)
            'kh': 'ኸ',  'khu': 'ኹ',  'khi': 'ኺ',
            'kha': 'ኻ', 'khe': 'ኼ',  'kh\'': 'ኽ',
            'kho': 'ኾ',

            # ወ family (w)
            'w': 'ወ',   'wu': 'ዉ',   'wi': 'ዊ',
            'wa': 'ዋ',  'we': 'ዌ',   'w\'': 'ው',
            'wo': 'ዎ',

            # ዐ family (aa)
            'aa': 'ዐ',  'aau': 'ዑ',  'aai': 'ዒ',
            'aaa': 'ዓ', 'aae': 'ዔ',  'aa\'': 'ዕ',
            'aao': 'ዖ',

            # ዘ family (z)
            'z': 'ዘ',   'zu': 'ዙ',   'zi': 'ዚ',
            'za': 'ዛ',  'ze': 'ዜ',   'z\'': 'ዝ',
            'zo': 'ዞ',  'zwa': 'ዟ',

            # ዠ family (zh)
            'zh': 'ዠ',  'zhu': 'ዡ',  'zhi': 'ዢ',
            'zha': 'ዣ', 'zhe': 'ዤ',  'zh\'': 'ዥ',
            'zho': 'ዦ', 'zhwa': 'ዧ',

            # የ family (y)
            'y': 'የ',   'yu': 'ዩ',   'yi': 'ዪ',
            'ya': 'ያ',  'ye': 'ዬ',   'y\'': 'ይ',
            'yo': 'ዮ',

            # ደ family (d)
            'd': 'ደ',   'du': 'ዱ',   'di': 'ዲ',
            'da': 'ዳ',  'de': 'ዴ',   'd\'': 'ድ',
            'do': 'ዶ',  'dwa': 'ዷ',

            # ጀ family (j)
            'j': 'ጀ',   'ju': 'ጁ',   'ji': 'ጂ',
            'ja': 'ጃ',  'je': 'ጄ',   'j\'': 'ጅ',
            'jo': 'ጆ',  'jwa': 'ጇ',

            # ገ family (g)
            'g': 'ገ',   'gu': 'ጉ',   'gi': 'ጊ',
            'ga': 'ጋ',  'ge': 'ጌ',   'g\'': 'ግ',
            'go': 'ጎ',  'gwa': 'ጐ',  'gwi': 'ጒ',
            'gwaa': 'ጓ','gwe': 'ጔ',  'gw': 'ጕ',

            # ጠ family (th)
            'th': 'ጠ',  'thu': 'ጡ',  'thi': 'ጢ',
            'tha': 'ጣ', 'the': 'ጤ',  'th\'': 'ጥ',
            'tho': 'ጦ', 'thwa': 'ጧ',

            # ጨ family (ch\')
            'ch\'': 'ጨ','ch\'u': 'ጩ','ch\'i': 'ጪ',
            'ch\'a': 'ጫ','ch\'e': 'ጬ','ch\'\'': 'ጭ',
            'ch\'o': 'ጮ','ch\'wa': 'ጯ',

            # ጰ family (ph)
            'ph': 'ጰ',  'phu': 'ጱ',  'phi': 'ጲ',
            'pha': 'ጳ', 'phe': 'ጴ',  'ph\'': 'ጵ',
            'pho': 'ጶ', 'phwa': 'ጷ',

            # ጸ family (ts)
            'ts': 'ጸ',  'tsu': 'ጹ',  'tsi': 'ጺ',
            'tsa': 'ጻ', 'tse': 'ጼ',  'ts\'': 'ጽ',
            'tso': 'ጾ', 'tswa': 'ጿ',

            # ፀ family (ts\')
            'ts\'': 'ፀ','ts\'u': 'ፁ','ts\'i': 'ፂ',
            'ts\'a': 'ፃ','ts\'e': 'ፄ','ts\'\'': 'ፅ',
            'ts\'o': 'ፆ',

            # ፈ family (f)
            'f': 'ፈ',   'fu': 'ፉ',   'fi': 'ፊ',
            'fa': 'ፋ',  'fe': 'ፌ',   'ff': 'ፍ',
            'fo': 'ፎ',  'fwa': 'ፏ',

            # ፐ family (p)
            'p': 'ፐ',   'pu': 'ፑ',   'pi': 'ፒ',
            'pa': 'ፓ',  'pe': 'ፔ',   'p\'': 'ፕ',
            'po': 'ፖ',  'pwa': 'ፗ',

            # Punctuation marks
            ':' : '፡',  # Word separator (hulet netib)
            ':-': '፤',  # Semi-colon
            '.' : '።',  # Full stop
            ':,': '፣',  # Comma
            ':!': '፥',  # Colon
            ':?': '፧',  # Question mark
            ':-:': '፦', # Preface colon
        }
        
        # For now, return the same mapping for all modes
        # In a more complete implementation, we could have different mappings
        return default_mapping

In [ ]:
#| export
class KeyboardHotkeyManager:
    """Manages hotkeys for controlling the Geez keyboard."""
    
    def __init__(self):
        self.hotkeys = {
            'toggle_input'       : ['ctrl+space'],  # Toggle between active and inactive
            'switch_to_tigrinya' : ['alt+t'     ],  # Switch to Tigrinya
            'switch_to_amharic'  : ['alt+a'     ],  # Switch to Amharic
            'switch_to_latin'    : ['alt+l'     ],    # Switch to Latin
            'cancel_buffer'      : ['escape'    ]      # Cancel current buffer
        }
    
    def register_hotkey(self, action: str, keys: List[str]):
        """Register a hotkey for a specific action."""
        self.hotkeys[action] = keys
    
    def get_hotkeys_for_action(self, action: str) -> List[str]:
        """Get the hotkeys registered for a specific action."""
        return self.hotkeys.get(action, [])

## Input Method Implementation

In [ ]:
#| export
@dataclass
class ProcessResult:
    """Result of processing a keystroke"""
    output_char: str = ""          # Character to output
    suppress_key: bool = False     # Whether to suppress the original key
    # backspace_count: int = 0       # Number of backspaces needed
    buffer_state: str = ""         # Current buffer state (for debugging)

In [ ]:
#| export
class GeezInputMethod:
    """An improved input method for typing in Ge'ez alphabets."""
    
    def __init__(self, config: Optional[KeyboardConfig] = None):
        """Initialize the input method with optional configuration."""
        self.config              = config or KeyboardConfig()
        self.mode                = self.config.default_mode
        self.mapping             = KeyboardMappings.get_mapping_for_mode(self.mode)
        self.buffer              = ""
        self.last_keystroke_time = 0
        self.sequence_timeout    = 2.0  # in seconds
        self.active              = True
        self.visible_chars       = 0
        # Pre-compute mapping structures for efficiency
        self._build_lookup_structures()
    
    def set_mode(self, mode: InputMode):
        """Set the input mode and update mappings accordingly."""
        self.mode = mode
        self.mapping = KeyboardMappings.get_mapping_for_mode(self.mode)
        
    def _build_lookup_structures(self):
        """Build efficient lookup structures from the mapping."""
        # Find all prefixes that could lead to longer sequences
        self.valid_prefixes = set()
        self.max_sequence_length = 0
        
        for combo in self.mapping.keys():
            self.max_sequence_length = max(self.max_sequence_length, len(combo))
            # Add all prefixes of this combo
            for i in range(1, len(combo) + 1):
                self.valid_prefixes.add(combo[:i])
        
        # Build a trie-like structure for efficient prefix checking
        self.prefix_tree = {}
        for combo, geez in self.mapping.items():
            node = self.prefix_tree
            for char in combo:
                if char not in node:
                    node[char] = {'children': {}, 'is_terminal': False, 'value': None}
                node = node[char]['children']
            # Mark the end of a valid sequence
            parent = self.prefix_tree
            for char in combo[:-1]:
                parent = parent[char]['children']
            parent[combo[-1]]['is_terminal'] = True
            parent[combo[-1]]['value'] = geez
    
    def _check_timeout(self) -> bool:
        """Check if the sequence has timed out."""
        current_time = time.time()
        if self.last_keystroke_time == 0:
            return False
        return (current_time - self.last_keystroke_time) > self.sequence_timeout
    
    def _find_longest_match(self, text: str) -> Tuple[Optional[str], int]:
        """Find the longest matching prefix in the text.
        Returns (matched_geez_char, match_length) or (None, 0) if no match."""
        for length in range(len(text), 0, -1):
            prefix = text[:length]
            if prefix in self.mapping:
                return self.mapping[prefix], length
        return None, 0
    
    def _has_potential_continuation(self, buffer: str) -> bool:
        """Check if the buffer could potentially lead to a valid sequence."""
        # Check if any mapping key starts with the current buffer
        # and is longer than the buffer
        for key in self.mapping.keys():
            if key.startswith(buffer) and len(key) > len(buffer):
                return True
        return False
    
    def process_keystroke(self, key: str) -> ProcessResult:
        """
        Process a single keystroke and return the result.
        
        This is the core algorithm that handles:
        1. Timeout detection and buffer flushing
        2. Character accumulation
        3. Eager matching with look-ahead
        4. Backspace calculation
        """
        result = ProcessResult()
        
        # Only process single alphabetic characters
        if not (len(key) == 1 and key.isalpha()) or not self.active:
            return result
        
        current_time = time.time()
        
        # Check for timeout on existing buffer
        if self.buffer and self._check_timeout():
            # Timeout occurred - try to output what we have
            geez_char, match_len = self._find_longest_match(self.buffer)
            if geez_char:
                # Output the old buffer match and start fresh with new key
                result.output_char = geez_char
                result.suppress_key = True
                # result.backspace_count = 0  # No backspace on timeout - buffer was already visible
                # IMPORTANT: Reset buffer with just the new key
                self.buffer = key.lower()
                self.last_keystroke_time = current_time
                self.visible_chars = 1  # New sequence started
                result.buffer_state = self.buffer
                return result
            else:
                # No match for old buffer, clear it and start fresh
                self.buffer = key.lower()
                self.last_keystroke_time = current_time
                self.visible_chars = 1
                result.suppress_key = True
                result.buffer_state = self.buffer
                return result
        
        # If buffer is empty, this is a new sequence
        if not self.buffer:
            self.visible_chars = 1
        else:
            self.visible_chars += 1
        
        # Add current key to buffer
        self.buffer += key.lower()
        self.last_keystroke_time = current_time
        result.suppress_key = True  # Always suppress while processing
        
        # Check if current buffer has an exact match
        if self.buffer in self.mapping:
            # We have an exact match
            if self._has_potential_continuation(self.buffer):
                # Could lead to longer match, wait for more input
                result.buffer_state = self.buffer
                return result
            else:
                # No longer match possible, output this match
                result.output_char = self.mapping[self.buffer]
                # Only backspace the visible chars minus the last one (which is suppressed)
                # result.backspace_count = self.visible_chars - 1
                # IMPORTANT: Clear the buffer completely after outputting
                self.buffer = ""
                self.last_keystroke_time = 0  # Reset timestamp
                self.visible_chars = 0
                result.buffer_state = self.buffer
                return result
        
        # No exact match, check if buffer could still lead to a match
        if self._has_potential_continuation(self.buffer):
            # Still potential for a match, keep accumulating
            result.buffer_state = self.buffer
            return result
        
        # No exact match and no potential continuation
        # Find the longest prefix that matches
        if len(self.buffer) > 1:
            # Try to find longest match in buffer minus last character
            prefix = self.buffer[:-1]
            geez_char, match_len = self._find_longest_match(prefix)
            
            if geez_char:
                # Output the match and keep only the unmatched portion in buffer
                result.output_char = geez_char
                # Calculate backspaces based on actual visible characters for this match
                # result.backspace_count = min(match_len - 1, self.visible_chars - 1)
                # Keep only the unmatched portion
                self.buffer = self.buffer[match_len:]
                self.visible_chars = len(self.buffer)  # Reset to remaining buffer length
                self.last_keystroke_time = current_time
                result.buffer_state = self.buffer
                return result
        
        # No matches at all - let the original character through
        # This happens when buffer is a single char with no matches or continuations
        result.suppress_key = False
        self.buffer = ""
        self.last_keystroke_time = 0
        self.visible_chars = 0  # Reset this too
        result.buffer_state = self.buffer
        return result
    
    def reset(self):
        """Reset the input buffer."""
        self.buffer              = ""
        self.last_keystroke_time = 0
        self.visible_chars       = 0
    
    def test_conversion(self, text: str, key_delay: float = 0.1):
        """Test method to verify conversions work with simulated delays."""
        print(f"\nTesting conversion for: '{text}' (key delay: {key_delay}s)")
        self.reset()
        
        results = []
        for i, char in enumerate(text):
            if key_delay > 0 and i > 0:
                time.sleep(key_delay)  # Simulate typing delay
            
            result = self.process_keystroke(char)
            results.append({
                'char': char,
                'output': result.output_char,
                'suppress': result.suppress_key,
                # 'backspace': result.backspace_count,
                'buffer': result.buffer_state
            })
        
        # Display results
        for i, r in enumerate(results):
            print(f"  Step {i+1}: '{r['char']}' -> ", end="")
            if r['output']:
                print(f"Output: '{r['output']}', ", end="")
            print(f"Suppress: {r['suppress']}, Buffer: '{r['buffer']}'")
        
        # Show final output
        output = ""
        for r in results:
            if r['output']:
                output += r['output']
            elif not r['suppress']:
                output += r['char']
        print(f"  Final output: '{output}'")
        return output

In [ ]:
#| export
class SystemInputHandler:
    """Handles system-wide keyboard input for the Geez keyboard."""
    
    def __init__(self, config_path: Optional[str] = None):
        """Initialize the input handler with optional configuration path."""
        self.input_method = GeezInputMethod()
        self.setup_keyboard_hook()

    def setup_keyboard_hook(self):
        """Set up system-wide keyboard hook using the keyboard library."""
        try:
            import keyboard
            
            def on_key(event):
                if event.event_type == keyboard.KEY_DOWN:
                    if len(event.name) == 1 and event.name.isalpha():
                        result = self.input_method.process_keystroke(event.name)
                        
                        if result.suppress_key:
                            # # Handle backspace
                            # if result.backspace_count > 0:
                            #     for _ in range(result.backspace_count):
                            #         keyboard.send('backspace')
                            
                            # Output the converted character
                            if result.output_char:
                                keyboard.write(result.output_char)
                            
                            # Suppress the original keystroke
                            return False
                
                # Let everything else through
                return True
            
            keyboard.hook(on_key, suppress=True)
            
        except ImportError:
            print("Keyboard library not available. Please install required dependencies.")

In [ ]:
#| export
class StatusIndicator:
    """A status indicator for the Geez keyboard."""
    
    def __init__(self, input_method: GeezInputMethod):
        """Initialize with the input method to monitor."""
        self.input_method = input_method
        self.visible = False
    
    def show(self, message: str, duration: float = 2.0):
        """Show the status indicator with a message."""
        # In a real implementation, this would show a GUI element
        print(f"STATUS: {message}")
        self.visible = True
    
    def hide(self):
        """Hide the status indicator."""
        self.visible = False

## Application Functions

In [ ]:
#| export
def start_keyboard(mode_str="tigrinya"):
    """Start the Geez keyboard with the specified input mode."""
    # Convert string mode to enum
    try:
        mode = InputMode(mode_str.lower())
    except ValueError:
        print(f"Invalid mode: {mode_str}. Using default mode (Tigrinya).")
        mode = InputMode.TIGRINYA
    
    # Create input handler with the specified mode
    config = KeyboardConfig(default_mode=mode)
    input_method = GeezInputMethod(config)
    input_handler = SystemInputHandler()
    input_handler.input_method = input_method  # Connect the input method
    input_handler.input_method.set_mode(mode)  # Set the correct mode
    
    # Show a status message
    status = StatusIndicator(input_method)
    status.show(f"Geez Keyboard started in {mode.value} mode")
    
    print(f"Geez Keyboard started in {mode.value} mode")
    print("Type to see Geez characters. Press Ctrl+C to exit.")
    
    return input_handler

In [ ]:
#| export
def main():
    """Command-line entry point for the Geez Keyboard."""
    parser = argparse.ArgumentParser(description="Geez Keyboard for typing in Tigrinya and Amharic")
    parser.add_argument("--mode", choices=["tigrinya", "amharic", "latin"], 
                        default="tigrinya", help="Input mode (default: tigrinya)")
    parser.add_argument("--test", action="store_true", help="Run conversion test")
    args = parser.parse_args()
    
    if args.test:
        # Test the conversion logic
        config = KeyboardConfig(default_mode=InputMode.TIGRINYA)
        input_method = GeezInputMethod(config)
        input_method.test_conversion("hello")
        input_method.test_conversion("ha")
        return
    
    keyboard_handler = start_keyboard(args.mode)
    
    print("Press Ctrl+C to exit")
    try:
        while True:
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("\nExiting Geez Keyboard")

# This will be used when the module is run directly
if __name__ == "__main__":
    main()

## Usage Examples

In [ ]:
# Basic usage example
input_handler = start_keyboard("tigrinya")

In [ ]:
# Configure with custom settings
config = KeyboardConfig(
    sequence_timeout = 0.7,  # Longer timeout for slower typing
    default_mode     = InputMode.TIGRINYA
)

In [ ]:
# Using custom mappings
custom_mapping = {
    'se': 'ሴ',
    'ez': 'ዝ',
    # Add your own custom mappings
}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()